In [ ]:
import csv
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import random

from scipy.special import logsumexp  # to deal with potential overflow issues


def set_seed(seed):  # For reproducibility, fix random seeds.
  random.seed(seed)
  np.random.seed(seed)

set_seed(42)

# Fashion MNIST

Download the Fashion MNIST dataset from [here](https://drive.google.com/drive/folders/1BnU7wVriolasZAZ1bSDTyll1Tp61hP1c?usp=sharing). It consists of 16 x 16 grayscale images (downsized from 28 x 28 for efficiency), split into 50,000 training and 10,000 validation images. Each image is labeled as one of 10 clothing categories (e.g., dress, sandal, shirt). We will assume that we have the directory `data/FashionMNIST/` in our Google Drive account. Let's load the data and stare at it.  

In [ ]:
# Load the Drive helper and mount. You will have to authorize this operation. 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
datadir = '/content/drive/My Drive/data/FashionMNIST/'
label_names = ['tshirt/top', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']  # Hardcoded

class FashionMNIST:

  def __init__(self, split):
    assert split in ['Tr', 'Vl']  # We don't have a test set
    self.inputs = np.load('{:s}x{:s}.npy'.format(datadir, split))  # (N, 16^2)
    self.labels = np.load('{:s}y{:s}.npy'.format(datadir, split))  # (N, 10)
    self.num_examples, self.dim = self.inputs.shape
    self.num_labels = self.labels.shape[1]

    # Partition data by labels
    self.partition = [self.inputs[np.where(np.argmax(self.labels, axis=1) == y)[0]] for y in range(self.num_labels)]

In [ ]:
data_train = FashionMNIST('Tr')
data_val = FashionMNIST('Vl')

print(data_train.inputs.shape, data_val.inputs.shape)

In [ ]:
def show_image(image, ax=None):
  width = int(np.sqrt(image.shape[0]))
  image = image.reshape(width, width)  
  if ax == None:
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
  imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
  imgplot.set_interpolation('nearest')
  ax.xaxis.set_ticks_position('top')
  ax.yaxis.set_ticks_position('left')
  plt.axis('off')

In [ ]:
for y in range(10):
  ax = plt.subplot(2, 5, y + 1)
  ax.clear()
  show_image(data_train.partition[y][0], ax)
  ax.set_title(label_names[y])

The images are a little blurry because we downsized them, but they will do. 

# Gaussian Mixture Model (GMM)

A **Gaussian mixture model** (GMM) with $K$ components has learnable parameters $\pi \in \mathbb{R}^K$ and $\mu_k \in \mathbb{R}^K, \Sigma_k \in \mathbb{R}^{K \times K}$ for each $k = 1 \ldots K$, where it is always assumed that 

- $\pi$ is a (full-support) distribution over the $K$ components. So $\pi_k > 0$ and $\sum_{k=1}^K \pi_k = 1$. 
- $\Sigma_k$ is a symmetric and positive definite (thus invertible) for each $k$. 

A GMM  defines a joint distribution over $k \in \{1 \ldots K\}$ and $x \in \mathbb{R}^d$ by 
$$
p(k,x) = \pi_k \times \mathcal{N}(\mu_k, \Sigma_k)(x)
$$
where $\mathcal{N}(\mu_k, \Sigma_k)(x)$ is the probability of $x$ under the Gaussian distribution with mean $\mu_k$ and covariance matrix $\Sigma_k$. For practical reasons, we do the following:  

- We optionally restrict ourselves to *diagonal* covariance matrices to make calculation simpler and more efficient.
- We work in *log space* for numerical stability, that is: $\log p(k,x) = \log \pi_k + \log \mathcal{N}(\mu_k, \Sigma_k)(x)$. 
- We make all variables multidimensional tensors so that we can use linear algebraic operations instead of for loops. 

In [ ]:
class GMM:

  def __init__(self, dim, num_components, diag=False):
    self.pi = np.full(num_components, 1. / num_components)  # (K,)
    self.mu = np.zeros((num_components, dim))  # (K, d)
    if diag:
      self.sigma = np.ones((num_components, dim))  # (K, d)
    else:   
      self.sigma = np.array([np.identity(dim) for _ in range(num_components)])   # (K, d, d)
    self.diag = diag

  def compute_log_probs(self, inputs):  # (N, d)
    log_pi = np.log(self.pi)[:, np.newaxis]  # (K, 1)
    diffs = inputs[np.newaxis, :, :] - self.mu[:, np.newaxis, :]  # (K, N, d)

    # TODO: implement, do not use for loops
    log_probs = None  # (K, N): log p(k, inputs[i])

    return log_probs  

  def compute_posteriors(self, inputs):  # (N, d)
    log_probs = self.compute_log_probs(inputs)  # (K, N): log p(k, x)
    marginal_log_probs = logsumexp(log_probs, axis=0)  # (N,): log p(x)
    marginal_log_likelihood = marginal_log_probs.mean()  # Scalar
    posteriors = np.exp(log_probs - marginal_log_probs[np.newaxis, :])  # (K, N): p(k|x)
    return posteriors, marginal_log_likelihood, marginal_log_probs

In [ ]:
import unittest

from scipy.stats import multivariate_normal 

class TestGMM(unittest.TestCase):
      
  def setUp(self):
    set_seed(42)
    self.dim = 100
    self.num_components = 7
    self.num_examples = 200
    self.power = 2 # Check for numerical stability

    self.inputs = np.random.randn(self.num_examples, self.dim)

  def test_model_diag(self): 
    model = self.init_model(diag=True)
    log_probs_gold = self.get_log_probs_gold(model)
    log_probs = model.compute_log_probs(self.inputs)
    for k in range(self.num_components):
      for i in range(self.num_examples):
        self.assertAlmostEqual(log_probs[k, i], log_probs_gold[k, i])

  def test_model_nondiag(self): 
    model = self.init_model(diag=False)
    log_probs_gold = self.get_log_probs_gold(model)
    log_probs = model.compute_log_probs(self.inputs)
    for k in range(self.num_components):
      for i in range(self.num_examples):
        self.assertAlmostEqual(log_probs[k, i], log_probs_gold[k, i])

  def get_log_probs_gold(self, model):
    log_probs_gold = []
    for k in range(self.num_components):
      dist = multivariate_normal(mean=model.mu[k], cov=(np.diag(model.sigma[k]) if model.diag else model.sigma[k]))
      log_probs_gold.append([np.log(model.pi[k]) + dist.logpdf(self.inputs[i]) for i in range(self.num_examples)])
    return np.array(log_probs_gold)

  def init_model(self, diag=False):
    model = GMM(self.dim, self.num_components, diag=diag)
    pi_unnormalized = np.random.uniform(size=(self.num_components,)) ** self.power
    model.pi = pi_unnormalized / pi_unnormalized.sum()
    model.mu = np.random.randn(self.num_components, self.dim) ** self.power 
    if diag:
      model.sigma = np.random.randn(self.num_components, self.dim) ** self.power
    else:
      model.sigma = np.array([np.diag(np.random.randn(self.dim)) ** self.power for _ in range(self.num_components)])
    return model
            
unittest.main(TestGMM(), argv=[''], verbosity=2, exit=False)

# Expectation Maximization (EM)

The EM algorithm trains a GMM on *unlabeled* data by alternating the E step and the M step:
- E step: Compute posteriors $p(k|x_i)$ for every training input $i = 1 \ldots N$ and label $k = 1\ldots K$. This needs an initial set of parameter values.
- M step: Calculate the maximum-likelihood estimate of parameters under the posteriors.


In [ ]:
class GMMTrainerEM:

  def __init__(self, model, smoothing=0.1):
    self.model = model
    self.smoothing = smoothing
    self.diag_smoother =  smoothing * np.array([np.identity(model.mu.shape[1]) for _ in range(model.mu.shape[0])])

  def train(self, inputs, num_iterations_max=40, verbose=False, init_method='naive'):
    self.init_centers(inputs, init_method=init_method)
    mll_previous = -np.inf 
    posteriors, mll, _ = self.model.compute_posteriors(inputs)  # E step
    for iteration in range(num_iterations_max): 
      self.update_parameters(inputs, posteriors)  # M step
      posteriors, mll, _ = self.model.compute_posteriors(inputs)  # E step
      if verbose:
        print('Iteration {:3d}:\t marginal log-likelihood {:10.4f}'.format(iteration + 1,  mll))
      if np.isclose(mll, mll_previous):
        break
      mll_previous = mll
    return mll, iteration

  def update_parameters(self, inputs, posteriors):
    expected_counts = posteriors.sum(axis=1) + self.smoothing # (K,)
    self.model.pi = expected_counts / expected_counts.sum()

    weighted_sums = posteriors @ inputs  # (K, d)
    self.model.mu = weighted_sums / expected_counts[:, np.newaxis]

    diffs = inputs[np.newaxis, :, :] - self.model.mu[:, np.newaxis, :]  # (K, N, d)
    diffs_weighted = posteriors[:, :, np.newaxis] * diffs  # (K, N, d)
    if self.model.diag:
      self.model.sigma = None  # TODO: Implement
    else:
      self.model.sigma = None  # TODO: Implement
    
  def init_centers(self, inputs, init_method='naive'):
    # Find K centers from the given input vectors (N, d) somehow.
    if init_method == 'naive':

      # TODO: Implement
      # Tip: try selecting a random input as the first center, then iteratively selecting argmax_i sum_{l<k} ||x_l - x_i||
      self.model.mu = None 

    else: 
      raise ValueError('Unknown init method: ' + init_method)

# Experiments with Diagonal GMMs

We can use GMMs for classification, by training a GMM for each input partition with the same label then at test time predicting the label corresponding to the GMM with highest *marginal* likelihood.

In [ ]:
def compute_accuracy(models, data):  # models[y]: GMM for label y
  log_probs_all = np.zeros((data.num_examples, data.num_labels))
  for y in range(data.num_labels):
    log_probs_all[:, y] = None  # TODO: Implement
  preds = np.argmax(log_probs_all, axis=1)
  acc = np.mean(preds == np.argmax(data.labels, axis=1)) * 100.
  return acc, preds

One cool thing is that each mean $\mu_k$ corresponding to component $k$ can be visualized. We will hypothesize that different components learn different representations of the same label.

In [ ]:
def show_means(models, y):
  num_components = len(models[y].pi)
  fig, axes = plt.subplots(1, num_components)

  for k in range(num_components):
    if num_components == 1:
      ax = axes
    else:
      ax = axes[k]
    show_image(models[y].mu[k], ax)
    ax.axis("off")
    if k == 0:
      ax.set_title(label_names[y] + "/" + str(k))
    else:
      ax.set_title("/" + str(k))
  return fig, axes

We're ready to train diagonal GMMs with various $K$ values. The training is pretty sensitive to the smoothing parameter so be careful. 

In [ ]:
set_seed(0)
smoothing = 0.1

for num_components in [1, 3, 5, 7]:
  print('***Training a diagonal GMM with K={:d} components***'.format(num_components))
  models = []
  for y in range(data_train.num_labels):
    model = GMM(data_train.dim, num_components, diag=True)
    trainer = GMMTrainerEM(model, smoothing=smoothing)
    models.append(model)
    mll, iteration = trainer.train(data_train.partition[y], num_iterations_max=40, verbose=False)
    print('Label {:d}: {:2d} iterations, final MLL {:10.3f}'.format(y, iteration, mll))
  acc_train, _ = compute_accuracy(models, data_train)
  acc_val, _ = compute_accuracy(models, data_val)
  print('K={:d}: acc train {:3.2f}, acc val {:3.2f}'.format(num_components, acc_train, acc_val))
  for y in range(10):
    show_means(models, y)
    plt.show()

## <font color=red>Question</font>
For GMMs with diagonal covariance matrices, report
- $K=1$: train acc, val acc
- $K=3$: train acc, val acc

## <font color=red>Answer</font>
write your answer here

# Experiments with Full-Covariance GMMs

Similarly we will train full-covariance GMMs for various $K$ values. Again, the training is pretty sensitive to the smoothing parameter. In fact, the marginal log-likelihood may take positive values (invalid!!!) due to numerical instability, but we can still do classification with the model.

In [ ]:
set_seed(0)
smoothing = 0.001

for num_components in [1, 3, 5, 7]:
  print('***Training a full-covariance GMM with K={:d} components***'.format(num_components))
  models = []
  for y in range(data_train.num_labels):
    model = GMM(data_train.dim, num_components, diag=False)
    trainer = GMMTrainerEM(model, smoothing=smoothing)
    models.append(model)
    mll, iteration = trainer.train(data_train.partition[y], num_iterations_max=40, verbose=False)
    print('Label {:d}: {:2d} iterations, final MLL {:10.3f}'.format(y, iteration, mll))
  acc_train, _ = compute_accuracy(models, data_train)
  acc_val, _ = compute_accuracy(models, data_val)
  print('K={:d}: acc train {:3.2f}, acc val {:3.2f}'.format(num_components, acc_train, acc_val))
  for y in range(10):
    show_means(models, y)
    plt.show()

## <font color=red>Question</font>
For GMMs with full covariance matrices, report
- $K=1$: train acc, val acc
- $K=3$: train acc, val acc

## <font color=red>Answer</font>
write your answer here


---

## <font color=red>Question</font>
Explore the model space with different settings: varying number of components across classes (required), covariance matrix restrictions (required), initialization of GMM, convergence criteria of training, etc
- Report the setting and val accuracy of your single **best model**.
- Describe (in detail) the **trend in accuracies** (train and val) across settings
- Describe (in detail) the **trend in visualized cluster** (displayed by the showMeans function) across settings


## <font color=red>Answer</font>
write your answer here